# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,37.44,44,0,6.91,US,1740249012
1,1,bilibino,68.0546,166.4372,-34.46,99,53,1.95,RU,1740249206
2,2,yellowknife,62.4560,-114.3525,2.41,92,100,11.50,CA,1740249207
3,3,wailua homesteads,22.0669,-159.3780,66.99,83,0,3.44,US,1740249208
4,4,grytviken,-54.2811,-36.5092,45.41,87,75,3.44,GS,1740249210


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
#%%capture --no-display

# Configure the map plot
humidityPlot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    tiles = "OSM",
    frame_height = 504,
    frame_width = 896,
    hover_cols = ["City","Humidity"]
    
)

# Display the map
humidityPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
# max temp: 27 C ~ 80.6 °F
# min temp: 21C ~ 69.8 °F 
# wind < 4.5
# cloudiness = 0

idealCityDF = city_data_df.loc[
    (city_data_df["Max Temp"] > 69.8) & (city_data_df["Max Temp"] < 80.6) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
idealCityDF = idealCityDF.dropna()

# Display sample data
print(len(idealCityDF))
idealCityDF.head()

8


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
184,184,ratnagiri,16.9833,73.3000,77.76,60,0,3.83,IN,1740249423
239,239,la'ie,21.6477,-157.9253,70.95,70,0,2.35,US,1740249652
283,283,haiku-pauwela,20.9219,-156.3051,75.22,77,0,1.99,US,1740249703
341,341,east london,-33.0153,27.9116,70.74,88,0,3.44,ZA,1740249772
405,405,allapalli,19.4167,80.0667,72.39,33,0,2.71,IN,1740250187


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealCityDF[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
184,ratnagiri,IN,16.9833,73.3000,60,
239,la'ie,US,21.6477,-157.9253,70,
283,haiku-pauwela,US,20.9219,-156.3051,77,
341,east london,ZA,-33.0153,27.9116,88,
405,allapalli,IN,19.4167,80.0667,33,
424,de aar,ZA,-30.6497,24.0123,35,
546,napili-honokowai,US,20.9708,-156.6705,80,
583,salalah,OM,17.0151,54.0924,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]
hotel_df

Starting hotel search
ratnagiri - nearest hotel: Vihar
la'ie - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
east london - nearest hotel: No hotel found
allapalli - nearest hotel: No hotel found
de aar - nearest hotel: No hotel found
napili-honokowai - nearest hotel: Kahana Falls
salalah - nearest hotel: Muscat International Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
184,ratnagiri,IN,16.9833,73.3000,60,Vihar
283,haiku-pauwela,US,20.9219,-156.3051,77,Inn At Mama's Fish House
546,napili-honokowai,US,20.9708,-156.6705,80,Kahana Falls
583,salalah,OM,17.0151,54.0924,78,Muscat International Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
#%%capture --no-display

# Configure the map plot
hotelPlot = hotel_df.hvplot.points(
    "Lng", 
    "Lat",
    geo=True,
    color= "City",
    alpha= 0.8,
    size = "Humidity",
    tiles = "OSM",
    frame_height = 504,
    frame_width = 896,
    hover_cols = ["Hotel Name", "Country"]

)

# Display the map
hotelPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)